In [ ]:
# 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


# 카테고리 ID
wadiz_category = { 287:"테크/가전",
                  288:"패션/잡화",
                  310:"홈/리빙",
                  311:"뷰티",
                  289:"푸드",
                  293:"출판",
                  316:"클래스/컨설팅",
                  296:"레저/아웃도어",
                  297:"스포츠/모빌리티",
                  294:"컬쳐/아티스트",
                  290:"캐릭터/굿즈",
                  308:"반려동물",
                  309:"베이비/키즈",
                  292:"게임/취미",
                  317:"여행/숙박",
                  295:"기부/캨페인",
                  312:"후원",
                  313:"모임"
                 }


# 카테고리 ID 주면 펀딩별로 ID 값 뽑아오기
def get_funding_list(category):
    driver = webdriver.Chrome(r"C:\chrome_driver\chromedriver.exe") # VS code
    # driver = webdriver.Chrome()    # 셀레늄 업데이트 되서 크롬드라이버 필요없어짐 - 주피터



    url = f'https://www.wadiz.kr/web/wreward/category/{category}?endYn=Y&order=recent'
    driver.get(url)
    
    # 스크롤 몇번할지 입력
    prev_height = driver.execute_script("return document. body.scrollHeight")
    
    while True:
        # 첫번째로 스크롤 내리기
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        # 시간대기
        time.sleep(2)

        # 현재높이 저장
        current_height = driver.execute_script("return document. body.scrollHeight")

        # 현재높이와 끝의 높이가 끝이면 탈출
        if current_height == prev_height:
            print('스크롤 끝',category)
            break

        # 업데이트해줘서 끝낼 수 있도록
        prev_height = current_height
        
    funding_cards = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'PreorderMainCard_root__1-vcs'))
        )
    funding_list = []

    for card in funding_cards:
        funding_info = card.find_element(By.CSS_SELECTOR, '.PreorderMainCard_item__2FrO0').text
        funding_link = card.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        funding_id = funding_link.split("/")[-1].split("?")[0]
        # 일단 id 만 추출
        funding_list.append(funding_id)

        print(card)
        time.sleep(1)

    # 창 닫기 추가?
    driver.quit()
    #driver.close()
    
    return funding_list

result = []

# 카테고리 지정 (어떤 카테고리로 할지)
for i in list(wadiz_category.keys()):
    print(i)
    category_id = get_funding_list(i)
    result.append([i,category_id])
    time.sleep(1)
    
# category_id = [[] for _ in range(len(wadiz_category))]

df = pd.DataFrame(result,columns=['카테고리','상품번호'])
df.to_csv('카테고리별_상품번호(전체).csv')

print("complete!!")